In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import numpy as np

# Import the API key.
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

import collections
collections.Iterable = collections.abc.Iterable

In [2]:
# Call CSV file to DF
city_data_df=pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,Malavalli,12.3833,77.0833,overcast clouds,67.84,94,100,3.33,IN
1,1,Saint-Philippe,-21.3585,55.7679,clear sky,65.79,77,10,6.96,RE
2,2,Mataura,-46.1927,168.8643,broken clouds,47.48,45,81,10.58,NZ
3,3,Butaritari,3.0707,172.7902,broken clouds,82.38,70,82,18.79,KI
4,4,Turayf,31.6725,38.6637,clear sky,68.25,52,0,6.91,SA


In [3]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country
3,3,Butaritari,3.0707,172.7902,broken clouds,82.38,70,82,18.79,KI
13,13,Kavaratti,10.5669,72.6420,overcast clouds,80.38,78,100,13.58,IN
29,29,Sabang,5.8933,95.3214,few clouds,81.64,76,18,21.07,ID
36,36,Kapaa,22.0752,-159.3190,light rain,80.92,79,100,14.97,US
42,42,West Bay,19.3667,-81.4167,few clouds,84.27,66,20,8.05,KY
46,46,Kahului,20.8947,-156.4700,broken clouds,84.22,79,75,20.71,US
47,47,Nieuw Nickerie,5.9500,-56.9833,overcast clouds,80.89,82,96,9.71,SR
52,52,Cabo San Lucas,22.8909,-109.9124,broken clouds,92.50,74,67,2.95,MX
53,53,Araouane,18.9048,-3.5265,clear sky,91.56,31,5,4.72,ML
63,63,Hithadhoo,-0.6000,73.0833,broken clouds,81.37,69,82,15.10,MV


In [5]:
#Check for empty values
print(len(preferred_cities_df))
preferred_cities_df.count()

110


City_ID                110
City                   110
Lat                    110
Lng                    110
Current Description    110
Max Temp               110
Humidity               110
Cloudiness             110
Wind Speed             110
Country                110
dtype: int64

In [6]:
#Drop empty values and create new DataFrame "Clean Travel Cities"
clean_travel_cities_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_travel_cities_df))
clean_travel_cities_df.head()

110


,City_ID,City,Lat,Lng,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country
3,3,Butaritari,3.0707,172.7902,broken clouds,82.38,70,82,18.79,KI
13,13,Kavaratti,10.5669,72.6420,overcast clouds,80.38,78,100,13.58,IN
29,29,Sabang,5.8933,95.3214,few clouds,81.64,76,18,21.07,ID
36,36,Kapaa,22.0752,-159.3190,light rain,80.92,79,100,14.97,US
42,42,West Bay,19.3667,-81.4167,few clouds,84.27,66,20,8.05,KY


In [7]:
# Create new DataFrame hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Butaritari,KI,82.38,3.0707,172.7902,
13,Kavaratti,IN,80.38,10.5669,72.6420,
29,Sabang,ID,81.64,5.8933,95.3214,
36,Kapaa,US,80.92,22.0752,-159.3190,
42,West Bay,KY,84.27,19.3667,-81.4167,
46,Kahului,US,84.22,20.8947,-156.4700,
47,Nieuw Nickerie,SR,80.89,5.9500,-56.9833,
52,Cabo San Lucas,MX,92.50,22.8909,-109.9124,
53,Araouane,ML,91.56,18.9048,-3.5265,
63,Hithadhoo,MV,81.37,-0.6000,73.0833,


In [8]:
#Search for hotels within 3000 meters.
params = {
    "radius": 3000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    #Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    #Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan



Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [9]:
# 7. Drop Empty Values
hotel_df = hotel_df.dropna(axis="index", how="any")
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Butaritari,KI,82.38,3.0707,172.7902,Isles Sunset Lodge
13,Kavaratti,IN,80.38,10.5669,72.6420,Green leaf
29,Sabang,ID,81.64,5.8933,95.3214,Hotel Citra
36,Kapaa,US,80.92,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
42,West Bay,KY,84.27,19.3667,-81.4167,Calypso Cove
46,Kahului,US,84.22,20.8947,-156.4700,Maui Seaside Hotel
47,Nieuw Nickerie,SR,80.89,5.9500,-56.9833,Ishwardat Inn
52,Cabo San Lucas,MX,92.50,22.8909,-109.9124,Hotel Tesoro Los Cabos
63,Hithadhoo,MV,81.37,-0.6000,73.0833,Scoop Guest House
65,Naze,JP,81.52,28.3667,129.4833,Shishime Hotel


In [12]:
# Create DataFrame called hotel_df to store hotel name along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Description","Lat","Lng"]].copy("clean_travel_cities_df")
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng
3,Butaritari,KI,82.38,broken clouds,3.0707,172.7902
13,Kavaratti,IN,80.38,overcast clouds,10.5669,72.6420
29,Sabang,ID,81.64,few clouds,5.8933,95.3214
36,Kapaa,US,80.92,light rain,22.0752,-159.3190
42,West Bay,KY,84.27,few clouds,19.3667,-81.4167
46,Kahului,US,84.22,broken clouds,20.8947,-156.4700
47,Nieuw Nickerie,SR,80.89,overcast clouds,5.9500,-56.9833
52,Cabo San Lucas,MX,92.50,broken clouds,22.8909,-109.9124
53,Araouane,ML,91.56,clear sky,18.9048,-3.5265
63,Hithadhoo,MV,81.37,broken clouds,-0.6000,73.0833


In [13]:
#Create a new empty column named "Hotel Name" to hotel_df 
hotel_df["Hotel_Name"]=""

hotel_df.head

<bound method NDFrame.head of              City Country  Max Temp Current Description      Lat       Lng  \
3      Butaritari      KI     82.38       broken clouds   3.0707  172.7902   
13      Kavaratti      IN     80.38     overcast clouds  10.5669   72.6420   
29         Sabang      ID     81.64          few clouds   5.8933   95.3214   
36          Kapaa      US     80.92          light rain  22.0752 -159.3190   
42       West Bay      KY     84.27          few clouds  19.3667  -81.4167   
..            ...     ...       ...                 ...      ...       ...   
648          Biak      ID     81.52       broken clouds  -0.9131  122.8766   
653         Metro      ID     80.02     overcast clouds  -5.1131  105.3067   
654        Cairns      AU     87.12       broken clouds -16.9167  145.7667   
661         Adrar      MR     88.32       broken clouds  20.5022  -10.0711   
672  Dunmore Town      BS     81.93           clear sky  25.5000  -76.6500   

    Hotel_Name  
3               

In [14]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel_Name,Hotel Name
3,Butaritari,KI,82.38,broken clouds,3.0707,172.7902,,Isles Sunset Lodge
13,Kavaratti,IN,80.38,overcast clouds,10.5669,72.6420,,Green leaf
29,Sabang,ID,81.64,few clouds,5.8933,95.3214,,Hotel Citra
36,Kapaa,US,80.92,light rain,22.0752,-159.3190,,Sheraton Kauai Coconut Beach Resort
42,West Bay,KY,84.27,few clouds,19.3667,-81.4167,,Calypso Cove


In [16]:
clean_hotel_df=hotel_df
clean_hotel_df["Hotel Name"]=""
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel_Name,Hotel Name
3,Butaritari,KI,82.38,broken clouds,3.0707,172.7902,,
13,Kavaratti,IN,80.38,overcast clouds,10.5669,72.6420,,
29,Sabang,ID,81.64,few clouds,5.8933,95.3214,,
36,Kapaa,US,80.92,light rain,22.0752,-159.3190,,
42,West Bay,KY,84.27,few clouds,19.3667,-81.4167,,
46,Kahului,US,84.22,broken clouds,20.8947,-156.4700,,
47,Nieuw Nickerie,SR,80.89,overcast clouds,5.9500,-56.9833,,
52,Cabo San Lucas,MX,92.50,broken clouds,22.8909,-109.9124,,
63,Hithadhoo,MV,81.37,broken clouds,-0.6000,73.0833,,
65,Naze,JP,81.52,scattered clouds,28.3667,129.4833,,


In [17]:
#Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
#Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

#Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

#Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
#Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))